In [180]:
file = open("parsed/19110105.txt", "r")
text = file.readlines()

In [2]:
import re

import requests
from bs4 import BeautifulSoup


from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.pdfpage import PDFPage
from io import BytesIO
import argparse


def SoupeURL(URL):
    req = requests.get (URL , headers={'User-Agent': 'Mozilla/5.0'})
    p = req.content
    B = BeautifulSoup (p , "lxml")
    return B


def Debaliser(string):
    ch = str(string)
    ch = ch.replace("<br/>","")
    ch = re.sub ("<.*?>" ," ",str (ch))
    ch = re.sub("^ *","",str(ch))
    ch = re.sub("^\s*","",str(ch))
    ch = ch.rstrip()
    return ch

"""
ça parcours entre les deux année avec comme limite un nombre de gazettes.
ça retourne la liste des chemins ou y'a les pdf
"""
def parcourir_entre_deux(annee1,annee2, nb=200):
    cpt = 0
    cpt = cpt + 1
    fichiers = []
    for annee in range(annee1,annee2):
        for mois in range(1,13):
            for jour in range(1,32):
                jour_f = str(jour)
                if(len(jour_f) == 1):
                    jour_f = "0"+jour_f
                mois_f = str(mois)
                if(len(mois_f) == 1):
                    mois_f = "0" + mois_f
                date = str(annee) + mois_f + jour_f
                url = "http://www.enap.justice.fr/ARCHIVE/"+date+".pdf"
                req = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
                if(req.status_code == 200):
                    f = open("downloads/"+date+".pdf","wb")
                    fichiers.append("downloads/"+date+".pdf")
                    f.write(req.content)
                if(len(fichiers) == nb):
                    return fichiers

def pdf2xt(path):
    rsrcmgr = PDFResourceManager()
    retstr = BytesIO()
    device = TextConverter(rsrcmgr, retstr)
    with open(path, "rb") as fp:  # open in 'rb' mode to read PDF bytes
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.get_pages(fp, check_extractable=True):
            interpreter.process_page(page)
        device.close()
        text = retstr.getvalue()
        retstr.close()
    return text

import os


if __name__ == '__main__':

    if not os.path.exists("parsed"):
        os.mkdir("parsed")
        print("Directory ", "parsed", " Created ")


    if not os.path.exists("downloads"):
        os.mkdir("downloads")
        print("Directory ", "downloads", " Created ")

    #Je teste sur un seul journal
    pdf_path = parcourir_entre_deux(1911, 1920, 1)[0]
    print(pdf_path)
    content = (pdf2xt(pdf_path))
    f = open("parsed/"+pdf_path.split("/")[-1].split(".")[0]+".txt","wb")
    f.write(content)

downloads/19110101.pdf


In [3]:
import re
def extract_arrets(file_path):
    arrets = []
    with open(file_path) as file:
        content = file.read()
        L = re.findall("« ((La Cour|Le Tribunal) ; (.*?))\. » (.*?) —", content)
        for i in L:
            arrets.append(i[0])
    return arrets


def extract_all(file_path):
    with open(file_path) as file:
        datas = []
        content = file.read()
        L = re.findall(
            "(((COUR|TRIBUNAL).*?) (Présidence de ((.*?).)) (Audience du ((\d+) (\w+) (\d+\.)))) (.*?) « ((La Cour|Le Tribunal) ; (.*?))\. » (.*?) —",
            content)
        for i in L:
            print("====")
            juridiction = (i[1])
            president = (i[5])
            date = (i[6]).replace("Audience du ","")
            arret = (i[14])
            datas.append((president,date,juridiction,arret))
        return datas

In [141]:
import pandas as pd
def extract(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        datas = []
        text = file.read()

    df = pd.DataFrame(columns=[ "arrêt", "date", "juridiction"])
    f = re.findall(r"(?<!DU)(?<!«)((?: TRIBUNAL| COUR).+?) du ((?:\S+ ){3})", str(text))
    s = re.split(r"(?<!DU)(?<!«)(?:(?: TRIBUNAL| COUR).+?) du (?:(?:\S+ ){3})", str(text))
    for (cour, date), string in zip(f, s[1:]):
        m = re.findall(r"((?:La Cour|Le Tribunal).+)", string)
        if len(m) > 0:
            m2 = re.findall(r"(.+)(?:»|OBS|■)", m[-1])
            if len(m2)> 0 :
                df = df.append({ 'arrêt' : m2[0] , "date": date, "juridiction" : cour},ignore_index=True)
            else:
                df = df.append({ 'arrêt' : m[-1] , "date": date, "juridiction" : cour},ignore_index=True)
    df["juridiction"] = df["juridiction"].apply(lambda x : re.split(r"\.|\(", x)[0] )
    df["arrêt"] = df["arrêt"].apply(lambda x : re.split(r"OBSERVATIONS", x)[0] )
    df["date"] = df["date"].apply(lambda x : re.split(r"\.", x)[0] )
    return df

In [138]:
def get_annee(annee):
    annee = str(annee)
    if not os.path.exists("cache/"):
        os.mkdir("cache/")
        print("Directory ", "cache/" , " Created ")
    if not os.path.exists("cache/"+annee):
        os.mkdir("cache/"+annee)
        print("Directory ", "cache/"+annee , " Created ")
    DF = pd.DataFrame(columns=[ "arrêt", "date", "juridiction", "lien", "id"])
    for mois in range(1, 2):
        for jour in range(2, 3):
            jour_f = str(jour)
            if (len(jour_f) == 1):
                jour_f = "0" + jour_f
            mois_f = str(mois)
            if (len(mois_f) == 1):
                mois_f = "0" + mois_f
            date = str(annee) + mois_f + jour_f
            url = "http://www.enap.justice.fr/ARCHIVE/" + date + ".pdf"
            print(annee, mois_f, jour_f)
            req = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
            if (req.status_code == 200):
                print(mois,"/",jour)
                f = open("tmp.bin", "wb")
                f.write(req.content)
                f.close()
                content = pdf2xt("tmp.bin")
                f = open("tmp.bin", "wb")
                f.write(content)
                f.close()
                df = extract("tmp.bin")
                df["lien"] = url
                df["id"] = "" + annee +mois_f+jour_f + df.index.map(str)
                DF = DF.append(df)
            else:
                url = "http://data.decalog.net/enap1/liens/Gazette/ENAP_GAZETTE_TRIBUNAUX_" + date + ".pdf"
                req = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
                if (req.status_code == 200):
                    print(mois,"/",jour)
                    f = open("tmp.bin", "wb")
                    f.write(req.content)
                    f.close()
                    content = pdf2xt("tmp.bin")
                    f = open("tmp.bin", "wb")
                    f.write(content)
                    f.close()
                    df = extract("tmp.bin")
                    df["lien"] = url
                    df["id"] = "" + annee +mois_f+jour_f + df.index.map(str)
                    DF = DF.append(df)
    DF.to_csv(f"cache/{annee}/Gazette_des_tribunaux.csv", encoding="utf-8", sep = ";")
    return DF


In [149]:
df = get_annee(1864)

Directory  cache/1864  Created 
1864 01 02
1 / 2


In [152]:
f = open("tmp.bin", "r" ,encoding="utf-8")

In [153]:
text = f.read()

In [58]:
import pandas as pd
df = pd.read_csv(f"cache/{1875}/Gazette_des_tribunaux.csv", encoding="utf-8", sep = ";")

In [181]:

df = pd.DataFrame(columns=[ "arrêt", "date", "juridiction"])
f = re.findall(r"(?<!DU)(?<!«)((?: TRIBUNAL| COUR).+?) du ((?:\S+ ){3})", str(text))
s = re.split(r"(?<!DU)(?<!«)(?:(?: TRIBUNAL| COUR).+?) du (?:(?:\S+ ){3})", str(text))
for (cour, date), string in zip(f, s[1:]):
    m = re.findall(r"(« (?:La (?:C|G)our|Le Tribunal).+?)(?:»|[A-Z]{3})", string)
    for message in m:
        df = df.append({ 'arrêt' : message , "date": date, "juridiction" : cour},ignore_index=True)
df["juridiction"] = df["juridiction"].apply(lambda x : re.split(r"\.|\(", x)[0] )
df["arrêt"] = df["arrêt"].apply(lambda x : re.split(r"OBSERVATIONS", x)[0] )
df["date"] = df["date"].apply(lambda x : re.split(r"\.", x)[0] )

In [192]:
df.arrêt[1]

"« La Cour ; « Considérant qu\\'au cours des années 1907\\' et 1908, Heurard de Fontgalland assurait d\\'abord contre l\\'incendie, puis contre le vol, un mobilier garnissant la villa Malgré tout, à Triel, louée par la demoiselle Lefèvre, sa maîtresse ; que, par des avenants succes-sifs,, il augmentait à diverses reprises le montant des capitaux assurés, de telle sorte que, s\\'il fallait en croire les déclarations conteaues dans ces actes, la valeur totale du mobilier aurait passé en dix-huit mois, de 20,000 à 35,000 francs ; que, dans cette somme, les tableaux et objets d\\'art portés, en 1907, pour 5,000 francs, figurent, en 1908, pour 12,000 francs-, et que les livres et documents précieux dont aucun n\\'est mentionné dans la police de 1907 comp-tent pour 5,000 francs dans l\\'avenant de 1908 ; que, dans le même laps de temps, l\\'argenterie passe de 500 à 2,000 francs, et les bijoux de 500 à 1,000 francs ; « Considérant qu\\'invité à s\\'expliquer sur les cau-ses d\\'un renchériss